In [4]:

import os
import gc
import sys
import glob
import copy
import numpy as np
import pandas as pd
import netCDF4 as nc
import multiprocessing as mp
from datetime import datetime, timedelta
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import matplotlib.ticker as mticker
# from PLUMBER2_VPD_common_utils import *


In [7]:
# Path of PLUMBER 2 dataset
PLUMBER2_path  = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/nc_files/"
CMIP6_path     = '/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/CMIP6/'

# site_names, IGBP_types, clim_types, model_names = load_default_list()

# ========================================= 1D curve ========================================
# model_list     = model_names['model_select_new']
dist_type      = 'Poisson' #'Poisson' # 'Linear', None
uncertain_type = 'UCRTN_bootstrap'

CMIP6_list   =  ['ACCESS-CM2', 'BCC-CSM2-MR', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'EC-Earth3', 'KACE-1-0-G', 'MIROC6',
                 'MIROC-ES2L', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0']

scenarios    = ['historical','ssp245']
region_names = ['east_AU', 'west_EU', 'north_Am']

region_name = region_names[2]
scenario    = scenarios[0]
region      = {'name':'north_Am'}
CMIP6_model = CMIP6_list[0]
model_in    = 'GFDL'


In [9]:

'''
bin_input, var_predict, bin_series: 1-d array
'''

# Read lat and lon
var_tmp         = pd.read_csv(f'{CMIP6_path}/lat_lon/CMIP6_DT_filtered_by_VPD_lat_lon_{scenario}_{CMIP6_model}_{region["name"]}.csv',na_values=[''], usecols=['lat','lon'])

# Read predict LH
var_predict     = pd.read_csv(f'{CMIP6_path}/predict/predicted_CMIP6_DT_filtered_by_VPD_Qle_{scenario}_{CMIP6_model}_{model_in}_{region["name"]}_{dist_type}.csv',
                           na_values=[''],usecols=[model_in])
var_tmp[model_in]= var_predict[model_in].values



In [12]:
np.unique(var_tmp['lat']) # 22
np.unique(var_tmp['lon']) # 32

array([-124.6875, -122.8125, -120.9375, -119.0625, -117.1875, -115.3125,
       -113.4375, -111.5625, -109.6875, -107.8125, -105.9375, -104.0625,
       -102.1875, -100.3125,  -98.4375,  -96.5625,  -94.6875,  -92.8125,
        -90.9375,  -89.0625,  -87.1875,  -85.3125,  -83.4375,  -81.5625,
        -79.6875,  -77.8125,  -75.9375,  -74.0625,  -72.1875,  -70.3125,
        -68.4375,  -66.5625])

In [29]:
var             = var_tmp.groupby(['lat','lon']).quantile([0.5])

In [30]:
var

GFDL
lat    lon                     
25.625 -113.4375 0.5  58.789076
       -111.5625 0.5  58.333193
       -109.6875 0.5  58.057961
       -107.8125 0.5  58.387494
       -105.9375 0.5  58.313052
...                         ...
51.875 -74.0625  0.5  58.046077
       -72.1875  0.5  58.306112
       -70.3125  0.5  58.327175
       -68.4375  0.5  58.377181
       -66.5625  0.5  58.375290

[560 rows x 1 columns]

In [ ]:
# Read EF_annual_hist
if model_in == 'CABLE':
    EF_annual_hist  = pd.read_csv(f'{CMIP6_path}/EF_annual_hist/CMIP6_DT_filtered_by_VPD_EF_annual_hist_{scenario}_{CMIP6_model}_{region["name"]}.csv',na_values=[''], usecols=['EF_annual_hist'])
    var_tmp['EF_annual_hist'] = EF_annual_hist['EF_annual_hist'].values

# Groupby
var             = var_tmp.groupby(['lat','lon']).mean()
print('var', var)
var_vals        = pd.Series(var[model_in].values, name=model_in)
var_vals.to_csv(f'./txt/CMIP6/thirty_year_mean_per_pixel/CMIP6_DT_filtered_by_VPD_Qle_30year_mean_{scenario}_{CMIP6_model}_{model_in}_{region["name"]}.csv')

if model_in == 'CABLE':
    var_EF_annual_hist = pd.Series(var['EF_annual_hist'].values, name='EF_annual_hist')
    var_EF_annual_hist.to_csv(f'./txt/CMIP6/thirty_year_mean_per_pixel/CMIP6_DT_filtered_by_VPD_EF_annual_hist_30year_mean_{scenario}_{CMIP6_model}_{model_in}_{region["name"]}.csv')
